In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

In [17]:

def get_data(ticker, start_date= '2000-01-01', end_date = pd.to_datetime('today').strftime('%Y-%m-%d'), interval='1wk', period=None):
    if period is not None:
        df = yf.download(ticker, period=period, interval=interval)
    else:
        df = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    window_size = 3
    log_returns = np.log(df['Adj Close'] / df['Adj Close'].shift(1))
    rolling_std_dev = log_returns.rolling(window=window_size).std()
    rolling_annualized_volatility = rolling_std_dev * np.sqrt(52)
    df['Volatility'] = rolling_annualized_volatility
    df['weekly_change'] = df['Adj Close'].pct_change() * 100
    return df

In [5]:
def get_volatility_range(df, num_std_devs = 1):
    curr_volatility = df['Volatility'][-1]
    std = df['Volatility'].std()
    return (curr_volatility - num_std_devs * std, curr_volatility + num_std_devs * std)

def filter_volatility_range(df, a, b):
    return df[(df['Volatility'] >= a) & (df['Volatility'] <= b)]


def get_bounds_for_running_prob(weekly_changes_within_volatility_range):
    bounds = {}
    for percentile in range(5, 50, 5):
        percentile = percentile / 100
        range_lower, range_upper = weekly_changes_within_volatility_range.quantile(percentile), weekly_changes_within_volatility_range.quantile(1-percentile)
        bounds[int((1-(percentile * 2)) * 100)] = (range_lower, range_upper)

    return [f"within {percent} percents, the weekly change is between {bound}" for percent, bound in bounds.items()]

def get_percent_for_bound(a,b, weekly_changes_within_volatility_range):
    return len(weekly_changes_within_volatility_range[(weekly_changes_within_volatility_range >= a) & (weekly_changes_within_volatility_range <= b)]) / len(weekly_changes_within_volatility_range) * 100



In [30]:
# Get the weekly changes within the recent volatility range
df = get_data('AAPL', '2024-03-22', interval='1m')
print(df)

[*********************100%%**********************]  1 of 1 completed

                                 Open        High         Low       Close  \
Datetime                                                                    
2024-03-22 09:30:00-04:00  171.860001  172.320007  171.500000  172.057495   
2024-03-22 09:31:00-04:00  172.065002  172.080002  171.720001  171.899994   
2024-03-22 09:32:00-04:00  171.899994  171.910004  171.500595  171.701996   
2024-03-22 09:33:00-04:00  171.699997  171.750000  171.210007  171.240005   
2024-03-22 09:34:00-04:00  171.236893  171.236893  170.800003  170.804993   
...                               ...         ...         ...         ...   
2024-03-22 15:55:00-04:00  172.289993  172.470001  172.279999  172.380005   
2024-03-22 15:56:00-04:00  172.375000  172.429993  172.330002  172.350006   
2024-03-22 15:57:00-04:00  172.339996  172.419998  172.330002  172.404999   
2024-03-22 15:58:00-04:00  172.409897  172.490005  172.389999  172.478897   
2024-03-22 15:59:00-04:00  172.479004  172.479996  172.240005  172.250000   